In [ ]:
import pandas as pd
import torch
from torch.utils.data import DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from transformers import get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from datasets import Dataset
import numpy as np

In [ ]:
data = pd.read_csv("/kaggle/input/tweet-dataset/Tweets Dataset.csv")
data.head()

In [ ]:
data['Party'] = data['Party'].map({'Democrat' : 0, 'Republican' : 1})

In [ ]:
train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

In [ ]:
# Convert to Hugging Face Dataset format
train_dataset = Dataset.from_pandas(train_data)
val_dataset = Dataset.from_pandas(val_data)

In [ ]:
# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

In [ ]:
# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["Tweet"], padding="max_length", truncation=True, max_length=128)

train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

In [ ]:
# Set the format for PyTorch
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'Party'])
val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'Party'])

In [ ]:
# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

In [ ]:
# Load the BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

In [ ]:
# Set up the optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=2e-5)
total_steps = len(train_loader) * 3  # 3 epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

In [ ]:
# Training loop
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

In [ ]:
best_val_accuracy = 0

for epoch in range(3):
    model.train()
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['Party'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Gradient clipping
        optimizer.step()
        scheduler.step()

    avg_train_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}, Average Training Loss: {avg_train_loss}")

    # Validation loop
    model.eval()
    val_preds, val_labels = [], []
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['Party'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            preds = torch.argmax(logits, dim=-1)

            val_preds.extend(preds.cpu().numpy())
            val_labels.extend(labels.cpu().numpy())

    val_accuracy = accuracy_score(val_labels, val_preds)
    print(f"Validation Accuracy: {val_accuracy}")

    # Save the best model to avoid overfitting
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        model.save_pretrained("/kaggle/working/bert")
        tokenizer.save_pretrained("/kaggle/working/token")

In [ ]:
from sklearn.metrics import accuracy_score

# Evaluate the model on the validation set
model.eval()  # Set the model to evaluation mode
val_preds, val_labels = [], []

with torch.no_grad():  # Disable gradient computation
    for batch in val_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['Party'].to(device)

        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        # Get predicted labels by finding the index with the highest logit value
        preds = torch.argmax(logits, dim=-1)

        # Store predictions and labels
        val_preds.extend(preds.cpu().numpy())
        val_labels.extend(labels.cpu().numpy())

# Calculate accuracy
val_accuracy = accuracy_score(val_labels, val_preds)
print(f"Validation Accuracy: {val_accuracy}")

In [ ]:
def load_model():
    model = BertForSequenceClassification.from_pretrained("/kaggle/working/bert")
    tokenizer = BertTokenizer.from_pretrained("/kaggle/working/token")
    model.to(device)
    model.eval()
    return model, tokenizer

In [ ]:
model, tokenizer = load_model()

In [ ]:
# Predict gender function (Male or Female)
def predict_party(comment, model, tokenizer, device):
    model.eval()
    encoding = tokenizer(comment, padding="max_length", truncation=True, max_length=128, return_tensors="pt")
    input_ids = encoding["input_ids"].to(device)
    attention_mask = encoding["attention_mask"].to(device)
    with torch.no_grad():
        outputs = model(input_ids, attention_mask)
        prediction = torch.argmax(outputs.logits, dim=1).cpu().item()
    return "Democrat" if prediction == 0 else "Republican"

In [ ]:
# Predict gender probability function (Male, Female probabilities)
def predict_prob(text, model, tokenizer, device, max_length=128):
    model.eval()
    encoding = tokenizer(text, return_tensors='pt', max_length=max_length, padding='max_length', truncation=True)
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    with torch.no_grad():
        outputs = model(input_ids, attention_mask)
        softmax = torch.nn.Softmax(dim=1)
        probabilities = softmax(outputs.logits).cpu().numpy()
    return probabilities

In [ ]:
# Predict instance function (Wrapper to call predict_gender_prob)
def predict_instance(text, model, tokenizer, device):
    preds = predict_gender_prob(text, model, tokenizer, device)
    # Return as dictionary with 'Male' and 'Female' probabilities
    return {
        "Democrat Probability": preds[0][0],  # Probability for Male (class 0)
        "Republican Probability": preds[0][1]  # Probability for Female (class 1)
    }

In [ ]:
# Example usage
text_sample = "Today, Senate Dems vote to #SaveTheInternet. Proud to support similar #NetNeutrality legislation here in the House… https://t.co/n3tggDLU1L"
predict_party(text_sample, model, tokenizer, device)

In [ ]:
txt = "Check out my op-ed on need for End Executive Overreach Act: The White House is crippling our economy https://t.co/XCmjLB8Qyd via @DCExaminer"
predict_party(txt, model, tokenizer, device)